<a href="https://colab.research.google.com/github/JamieGal/GeoLit_KG-BERT-T/blob/master/KG_BERT_T_best_model_test_run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KG-BERT-T best model

Prior to running, please confirm path to folder containing saved model in section 5.

This will run the model test on the saved model for paramters;

Dataset: GeoLit-Long
Batch size: 32
Learning rate: 2e-5
Epochs: 5



In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

In [ ]:
!pip install pytorch
!pip install transformers

In [ ]:
from __future__ import absolute_import, division, print_function

import argparse
import csv
import glob ###
import logging
import os
import random
import json ###
import sys
import time
import datetime
from datetime import timedelta

import numpy as np
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
from torch.utils.data.distributed import DistributedSampler

try:
    from torch.utils.tensorboard import SummaryWriter
except:
    from tensorboardX import SummaryWriter 

from tqdm import tqdm, trange

from torch.nn import CrossEntropyLoss, MSELoss
from scipy.stats import pearsonr, spearmanr
from sklearn.metrics import matthews_corrcoef, f1_score
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

from transformers import BertForSequenceClassification, BertTokenizer, AdamW, get_linear_schedule_with_warmup, BertConfig, WEIGHTS_NAME

from transformers import glue_output_modes as output_modes

logger = logging.getLogger(__name__)

In [ ]:
# Functions from KG-BERT

class InputExample(object):

    def __init__(self, guid, text_a, text_b=None, text_c=None, label=None):
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.text_c = text_c
        self.label = label

class InputFeatures(object):
 
     def __init__(self, input_ids, input_mask, segment_ids, label_id):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id


class DataProcessor(object):
    """Base class for data converters for sequence classification data sets.
    Inputs for train and validation dataset and labels."""

    def get_train_examples(self, data_dir):
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        raise NotImplementedError()

    def get_labels(self, data_dir):
        raise NotImplementedError()

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, "r", encoding="utf-8") as f:
            reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
            lines = []
            for line in reader:
                if sys.version_info[0] == 2:
                    line = list(unicode(cell, 'utf-8') for cell in line)
                lines.append(line)
            return lines


class KGProcessor(DataProcessor):
    """Processor for knowledge graph data set."""
    def __init__(self):
        self.labels = set()
    
    def get_train_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "train.tsv")), "train", data_dir)

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev", data_dir)

    def get_test_examples(self, data_dir):
      """See base class."""
      return self._create_examples(
          self._read_tsv(os.path.join(data_dir, "test.tsv")), "test", data_dir)

    def get_relations(self, data_dir):
        """Gets all labels (relations) in the knowledge graph."""
        # return list(self.labels)
        with open(os.path.join(data_dir, "relations.txt"), 'r') as f:
            lines = f.readlines()
            relations = []
            for line in lines:
                relations.append(line.strip())
        return relations

    def get_labels(self, data_dir):
        """Gets all labels (0, 1) for triples in the knowledge graph."""
        return ["0", "1"]

    def get_entities(self, data_dir):
        """Gets all entities in the knowledge graph."""
        # return list(self.labels)
        with open(os.path.join(data_dir, "entities.txt"), 'r') as f:
            lines = f.readlines()
            entities = []
            for line in lines:
                entities.append(line.strip())
        return entities

    def get_train_triples(self, data_dir):
        """Gets training triples."""
        return self._read_tsv(os.path.join(data_dir, "train.tsv"))

    def get_dev_triples(self, data_dir):
        """Gets validation triples."""
        return self._read_tsv(os.path.join(data_dir, "dev.tsv"))

    def get_test_triples(self, data_dir):
        """Gets test triples."""
        return self._read_tsv(os.path.join(data_dir, "test.tsv"))

    def _create_examples(self, lines, set_type, data_dir):
        """Creates examples for the training and dev sets."""
        # entity to text
        ent2text = {}
        with open(os.path.join(data_dir, "entity2text.txt"), 'r') as f:
            ent_lines = f.readlines()
            for line in ent_lines:
                temp = line.strip().split('\t')
                if len(temp) == 2:
                    end = temp[1]#.find(',')
                    ent2text[temp[0]] = temp[1]#[:end]
  
        """Replace entities with entity2textlong descriptions.
        Hash out if using just entities."""
        with open(os.path.join(data_dir, "entity2textlong.txt"), 'r') as f:
            ent_lines = f.readlines()
            for line in ent_lines:
                temp = line.strip().split('\t')
                """Unhash if changing number of sentences."""
                #first_sent_end_position = temp[1].find(".")
                ent2text[temp[0]] = temp[1]#[:first_sent_end_position + 1] 

        entities = list(ent2text.keys())

        rel2text = {}
        with open(os.path.join(data_dir, "relation2text.txt"), 'r') as f:
            rel_lines = f.readlines()
            for line in rel_lines:
                temp = line.strip().split('\t')
                rel2text[temp[0]] = temp[1]      

        lines_str_set = set(['\t'.join(line) for line in lines])
        examples = []
        for (i, line) in enumerate(lines):
            
            head_ent_text = ent2text[line[0]]
            tail_ent_text = ent2text[line[2]]
            relation_text = rel2text[line[1]]

            if set_type == "dev" or set_type == "test":
                label = "1"
                guid = "%s-%s" % (set_type, i)
                text_a = head_ent_text
                text_b = relation_text
                text_c = tail_ent_text 
                self.labels.add(label)
                examples.append(
                    InputExample(guid=guid, text_a=text_a, text_b=text_b, text_c = text_c, label=label))
                
            elif set_type == "train":
                guid = "%s-%s" % (set_type, i)
                text_a = head_ent_text
                text_b = relation_text
                text_c = tail_ent_text 
                examples.append(
                    InputExample(guid=guid, text_a=text_a, text_b=text_b, text_c = text_c, label="1"))

                """Corrupt head or tail depending upon random number above or below 0.5."""
                rnd = random.random()
                guid = "%s-%s" % (set_type + "_corrupt", i)
                if rnd <= 0.5:
                    # corrupting head
                    for j in range(5):
                        tmp_head = ''
                        while True:
                            tmp_ent_list = set(entities)
                            tmp_ent_list.remove(line[0])
                            tmp_ent_list = list(tmp_ent_list)
                            tmp_head = random.choice(tmp_ent_list)
                            tmp_triple_str = tmp_head + '\t' + line[1] + '\t' + line[2]
                            if tmp_triple_str not in lines_str_set:
                                break                    
                        tmp_head_text = ent2text[tmp_head]
                        examples.append(
                            InputExample(guid=guid, text_a=tmp_head_text, text_b=text_b, text_c = text_c, label="0"))       
                else:
                    # corrupting tail
                    tmp_tail = ''
                    for j in range(5):
                        while True:
                            tmp_ent_list = set(entities)
                            tmp_ent_list.remove(line[2])
                            tmp_ent_list = list(tmp_ent_list)
                            tmp_tail = random.choice(tmp_ent_list)
                            tmp_triple_str = line[0] + '\t' + line[1] + '\t' + tmp_tail
                            if tmp_triple_str not in lines_str_set:
                                break
                        tmp_tail_text = ent2text[tmp_tail]
                        examples.append(
                            InputExample(guid=guid, text_a=text_a, text_b=text_b, text_c = tmp_tail_text, label="0"))                                                  
        return examples

def convert_examples_to_features(examples, label_list, max_seq_length, tokenizer, print_info = True):
    """Loads a data file into a list of `InputBatch`s."""

    label_map = {label : i for i, label in enumerate(label_list)}

    features = []
    for (ex_index, example) in enumerate(examples):
        if ex_index % 10000 == 0 and print_info:
            logger.info("Writing example %d of %d" % (ex_index, len(examples)))

        tokens_a = tokenizer.tokenize(example.text_a)

        tokens_b = None
        tokens_c = None

        """Add special tokens"""
        if example.text_b and example.text_c:
            tokens_b = tokenizer.tokenize(example.text_b)
            tokens_c = tokenizer.tokenize(example.text_c)
            _truncate_seq_triple(tokens_a, tokens_b, tokens_c, max_seq_length - 4)
        else:
            if len(tokens_a) > max_seq_length - 2:
                tokens_a = tokens_a[:(max_seq_length - 2)]

        tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens += tokens_b + ["[SEP]"]
            segment_ids += [1] * (len(tokens_b) + 1)
        if tokens_c:
            tokens += tokens_c + ["[SEP]"]
            segment_ids += [0] * (len(tokens_c) + 1)        

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        """Input mask for tokens is 1, and for padding is 0."""
        input_mask = [1] * len(input_ids)

        """Zero-pad up to the sequence length."""
        padding = [0] * (max_seq_length - len(input_ids))
        input_ids += padding
        input_mask += padding
        segment_ids += padding

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length

        label_id = label_map[example.label]

        if ex_index < 5 and print_info:
            logger.info("*** Example ***")
            logger.info("guid: %s" % (example.guid))
            logger.info("tokens: %s" % " ".join(
                    [str(x) for x in tokens]))
            logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
            logger.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
            logger.info(
                    "segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
            logger.info("label: %s (id = %d)" % (example.label, label_id))

        features.append(
                InputFeatures(input_ids=input_ids,
                              input_mask=input_mask,
                              segment_ids=segment_ids,
                              label_id=label_id))
    return features


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

def _truncate_seq_triple(tokens_a, tokens_b, tokens_c, max_length):
    while True:
        total_length = len(tokens_a) + len(tokens_b) + len(tokens_c)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b) and len(tokens_a) > len(tokens_c):
            tokens_a.pop()
        elif len(tokens_b) > len(tokens_a) and len(tokens_b) > len(tokens_c):
            tokens_b.pop()
        elif len(tokens_c) > len(tokens_a) and len(tokens_c) > len(tokens_b):
            tokens_c.pop()
        else:
            tokens_c.pop()


def format_time(elapsed):
    """Takes a time in seconds and returns a string hh:mm:ss"""
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

def simple_accuracy(preds, labels):
    return (preds == labels).mean()
  
def compute_metrics(preds, labels):
    return {"acc": simple_accuracy(preds, labels)}

In [ ]:
# THIS WILL RUN THE TEST AND LINK PREDICTION ON THE SAVED BEST MODEL

# Add path to best model folder in colab before running.

path_to_best_model_folder = "/content/drive/My Drive/PROJECT/KG_BERT_T_best_model"


data_dir = path_to_best_model_folder + "/geolit_data"
model_type = "bert"
model_name_or_path = "bert-base-uncased"
task_name = "kg"
output_dir = path_to_best_model_folder + "/geolit_output"

max_seq_length = 100
train_batch_size = 32
eval_batch_size = 300
learning_rate = 2e-5
num_train_epochs = 5.0

local_rank = -1
seed = 42
gradient_accumulation_steps = 1
loss_scale = 0 
processors = {"kg": KGProcessor,}


#Check GPU availability
if torch.cuda.is_available():    
# Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")    
n_gpu = torch.cuda.device_count()

# Setup logging
logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO if local_rank in [-1, 0] else logging.WARN)

# Set seed
#seed = 42 #
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if n_gpu > 0:
    torch.cuda.manual_seed_all(seed)

results = {}
model_results = []
processor = processors[task_name]()
label_list = processor.get_labels(data_dir) 
num_labels = len(label_list)

entity_list = processor.get_entities(data_dir)

#test_task = task_name
saved_model = path_to_best_model_folder

# Load pretrained model and tokenizer
model = BertForSequenceClassification.from_pretrained(saved_model, num_labels=num_labels)
tokenizer = BertTokenizer.from_pretrained(saved_model, do_lower_case=True) 
model.to(device) 

train_triples = processor.get_train_triples(data_dir)
dev_triples = processor.get_dev_triples(data_dir)
test_triples = processor.get_test_triples(data_dir)
all_triples = train_triples + dev_triples + test_triples

all_triples_str_set = set()
for triple in all_triples:
    triple_str = '\t'.join(triple)
    all_triples_str_set.add(triple_str)

test_dataset = processor.get_test_examples(data_dir)
test_features = convert_examples_to_features(
    test_dataset, label_list, max_seq_length, tokenizer)
logger.info("***** Running Prediction *****")
logger.info("  Num examples = %d", len(test_dataset))
logger.info("  Batch size = %d", eval_batch_size)
all_input_ids = torch.tensor([f.input_ids for f in test_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in test_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in test_features], dtype=torch.long)
all_label_ids = torch.tensor([f.label_id for f in test_features], dtype=torch.long)

eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
eval_sampler = SequentialSampler(eval_data)
eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=eval_batch_size)  

# Set model to eval, stops dropout
model.eval()

nb_test_steps = 0
eval_loss = 0.0
preds = None
out_label_ids = None
for batch in tqdm(eval_dataloader, desc="Testing"):
    model.eval()
    batch = tuple(t.to(device) for t in batch)

    with torch.no_grad():
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[3]}
        if model_type != 'distilbert':
            inputs['token_type_ids'] = batch[2] if model_type in ['bert'] else None
        outputs = model(**inputs)
        tmp_eval_loss, logits = outputs[:2]

        eval_loss += tmp_eval_loss.mean().item()
    nb_test_steps += 1
    
    if preds is None:
        preds = logits.detach().cpu().numpy()
        out_label_ids = inputs['labels'].detach().cpu().numpy()
    else:
        preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
        out_label_ids = np.append(out_label_ids, inputs['labels'].detach().cpu().numpy(), axis=0)

eval_loss = eval_loss / nb_test_steps
preds = np.argmax(preds, axis=1)
result = compute_metrics(preds, out_label_ids)
results.update(result)

result['eval_loss'] = eval_loss       


# ========================================
#              Link prediction
# ======================================== 

# Link prediction, from KG-BERT

# Collect ranks, rank left and rank right
ranks = []
ranks_left = []
ranks_right = []

# Collect hits left and right
hits_left = []
hits_right = []
hits = []

# Collect test triple rank left and right in order
trip_rank = []
trip_count = 0

# Hits@10 count
top_ten_hit_count = 0

# If Hits in top 10, append
for i in range(10):
    hits_left.append([])
    hits_right.append([])
    hits.append([])

# Collect triples
for test_triple in test_triples:
    head = test_triple[0]
    relation = test_triple[1]
    tail = test_triple[2]
    trip_count += 1

# Corrupt head will all entities (except if they exist in dataset)
    head_corrupt_list = [test_triple]
    for corrupt_ent in entity_list:
        if corrupt_ent != head:
            tmp_triple = [corrupt_ent, relation, tail]
            tmp_triple_str = '\t'.join(tmp_triple)
            if tmp_triple_str not in all_triples_str_set:
                head_corrupt_list.append(tmp_triple)

# Run model as above
    tmp_examples = processor._create_examples(head_corrupt_list, "test", data_dir)
    print(len(tmp_examples))
    tmp_features = convert_examples_to_features(tmp_examples, label_list, max_seq_length, tokenizer, print_info = False)
    all_input_ids = torch.tensor([f.input_ids for f in tmp_features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in tmp_features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in tmp_features], dtype=torch.long)
    all_label_ids = torch.tensor([f.label_id for f in tmp_features], dtype=torch.long)

    eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
    eval_sampler = SequentialSampler(eval_data)
    eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=eval_batch_size)

    model.eval()
    preds = None       
    out_label_ids = None
    for batch in tqdm(eval_dataloader, desc="Testing"):
        batch = tuple(t.to(device) for t in batch)
        with torch.no_grad():
            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'labels':         batch[3]}
            if model_type != 'distilbert':
                inputs['token_type_ids'] = batch[2] if model_type in ['bert', 'xlnet'] else None  
            outputs = model(**inputs)
            tmp_eval_loss, logits = outputs[:2]
        if preds is None:
            preds = logits.detach().cpu().numpy()
            out_label_ids = inputs['labels'].detach().cpu().numpy()
        else:
            preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
            out_label_ids = np.append(out_label_ids, inputs['labels'].detach().cpu().numpy(), axis=0) 

# Collect tensor predictions, and link to label tensor to ranking
# This sorts and ranks predictions and returns rank for left (head)
    rel_values = preds[:, all_label_ids[0]]
    rel_values = torch.tensor(rel_values)
    _, argsort1 = torch.sort(rel_values, descending=True)
    argsort1 = argsort1.cpu().numpy()
    rank1 = np.where(argsort1 == 0)[0][0]
    print('left: ', rank1)
    ranks.append(rank1+1)
    ranks_left.append(rank1+1)
    if rank1 < 10:
        top_ten_hit_count += 1

# As above, corrupt tail and run model
    tail_corrupt_list = [test_triple]
    for corrupt_ent in entity_list:
        if corrupt_ent != tail:
            tmp_triple = [head, relation, corrupt_ent]
            tmp_triple_str = '\t'.join(tmp_triple)
            if tmp_triple_str not in all_triples_str_set:
                tail_corrupt_list.append(tmp_triple)

    tmp_examples = processor._create_examples(tail_corrupt_list, "test", data_dir)
    tmp_features = convert_examples_to_features(tmp_examples, label_list, max_seq_length, tokenizer, print_info = False)
    all_input_ids = torch.tensor([f.input_ids for f in tmp_features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in tmp_features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in tmp_features], dtype=torch.long)
    all_label_ids = torch.tensor([f.label_id for f in tmp_features], dtype=torch.long)

    eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
    eval_sampler = SequentialSampler(eval_data)
    eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=eval_batch_size)

    model.eval()
    preds = None       
    out_label_ids = None
    for batch in tqdm(eval_dataloader, desc="Testing"):
        batch = tuple(t.to(device) for t in batch)
        with torch.no_grad():
            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'labels':         batch[3]}
            if model_type != 'distilbert':
                inputs['token_type_ids'] = batch[2] if model_type in ['bert', 'xlnet'] else None  # XLM, DistilBERT and RoBERTa don't use segment_ids
            outputs = model(**inputs)
            tmp_eval_loss, logits = outputs[:2]
        if preds is None:
            preds = logits.detach().cpu().numpy()
            out_label_ids = inputs['labels'].detach().cpu().numpy()
        else:
            preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
            out_label_ids = np.append(out_label_ids, inputs['labels'].detach().cpu().numpy(), axis=0) 

# Collect tensor predictions, and link to label tensor to ranking
# This sorts and ranks predictions and returns rank for left (head)            
    rel_values = preds[:, all_label_ids[0]]
    rel_values = torch.tensor(rel_values)
    _, argsort1 = torch.sort(rel_values, descending=True)
    argsort1 = argsort1.cpu().numpy()
    rank2 = np.where(argsort1 == 0)[0][0]
    ranks.append(rank2+1)
    ranks_right.append(rank2+1)
    print('right: ', rank2)
    print('mean rank until now: ', np.mean(ranks))
    if rank2 < 10:
        top_ten_hit_count += 1
    hits10 = top_ten_hit_count * 1.0 / len(ranks)
    print("hit@10 until now: ", top_ten_hit_count * 1.0 / len(ranks))

# Count hits 
    for hits_level in range(10):
        if rank1 <= hits_level:
            hits[hits_level].append(1.0)
            hits_left[hits_level].append(1.0)
        else:
            hits[hits_level].append(0.0)
            hits_left[hits_level].append(0.0)

        if rank2 <= hits_level:
            hits[hits_level].append(1.0)
            hits_right[hits_level].append(1.0)
        else:
            hits[hits_level].append(0.0)
            hits_right[hits_level].append(0.0)

# Collect hits at 1 and 3
    hits1 = np.mean(hits[0])
    hits3 = np.mean(hits[2])

# Collect triple number, left (head) rank and right (tail) rank
    trip_rank.append({'Triple': trip_count, 'Head entity': rank1, 'Tail entity': rank2})

# Logging metrics
    for i in [0,2,9]:
        logger.info('Hits left @{0}: {1}'.format(i+1, np.mean(hits_left[i])))
        logger.info('Hits right @{0}: {1}'.format(i+1, np.mean(hits_right[i])))
        logger.info('Hits @{0}: {1}'.format(i+1, np.mean(hits[i])))
    logger.info('Mean rank left: {0}'.format(np.mean(ranks_left)))
    logger.info('Mean rank right: {0}'.format(np.mean(ranks_right)))
    logger.info('Mean rank: {0}'.format(np.mean(ranks)))
    logger.info('Mean reciprocal rank left: {0}'.format(np.mean(1./np.array(ranks_left))))
    logger.info('Mean reciprocal rank right: {0}'.format(np.mean(1./np.array(ranks_right))))
    logger.info('Mean reciprocal rank: {0}'.format(np.mean(1./np.array(ranks))))

# Collect model results
model_results.append({
    'Batch size': train_batch_size,
    'Learning rate': learning_rate,
    'Epochs': num_train_epochs,
    'Mean rank': np.mean(ranks),
    'Mean reciprocal rank': np.mean(1./np.array(ranks)),
    'Hits@10': hits10,
    'Mean left': np.mean(ranks_left),
    'Mean right': np.mean(ranks_right),
    'Hits@3': hits3,
    'Hits@1': hits1})

print(model_results)